# Funciones de Ventana (Window Functions)

## Objetivos de Aprendizaje
- Comprender qué son las funciones de ventana y cuándo usarlas
- Aplicar funciones de ranking: ROW_NUMBER, RANK, DENSE_RANK
- Usar funciones de navegación: LAG, LEAD, FIRST, LAST
- Calcular agregaciones con ventanas: SUM, AVG, COUNT
- Definir frames de ventana personalizados

## Prerequisitos
- `06_spark_processing/04_joins_aggregations.ipynb`

## Tiempo Estimado
⏱️ 45 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 9: Análisis avanzado con funciones analíticas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import date

spark = SparkSession.builder.appName("WindowFunctions").getOrCreate()
spark.sparkContext.setLogLevel("WARN")

In [ ]:
# Datos de ventas para ejemplos
ventas = spark.createDataFrame([
    ("Norte", "Ana", date(2024, 1, 1), 15000),
    ("Norte", "Ana", date(2024, 1, 2), 18000),
    ("Norte", "Ana", date(2024, 1, 3), 12000),
    ("Norte", "Carlos", date(2024, 1, 1), 20000),
    ("Norte", "Carlos", date(2024, 1, 2), 22000),
    ("Norte", "Carlos", date(2024, 1, 3), 19000),
    ("Sur", "Maria", date(2024, 1, 1), 25000),
    ("Sur", "Maria", date(2024, 1, 2), 28000),
    ("Sur", "Maria", date(2024, 1, 3), 30000),
    ("Sur", "Juan", date(2024, 1, 1), 16000),
    ("Sur", "Juan", date(2024, 1, 2), 17000),
    ("Sur", "Juan", date(2024, 1, 3), 18000)
], ["region", "vendedor", "fecha", "monto"])

print("Datos de ventas:")
ventas.show()

---
# === SECCIÓN 1 ===
## 1. Introducción a Window Functions

### Explicación Conceptual
Las **funciones de ventana** calculan valores sobre un conjunto de filas relacionadas con la fila actual, SIN reducir el número de filas (a diferencia de GROUP BY).

**Analogía:** Imagina que estás en una carrera. GROUP BY te diría "el tiempo promedio de todos". Una función de ventana te diría "tu tiempo comparado con los demás" o "tu posición en el ranking", manteniendo tu fila individual.

**Componentes de una ventana:**
1. **PARTITION BY**: Divide los datos en grupos (como GROUP BY pero sin colapsar)
2. **ORDER BY**: Define el orden dentro de cada partición
3. **Frame**: Define qué filas incluir en el cálculo (opcional)

In [ ]:
# Definir una ventana
# Window.partitionBy() divide por grupos
# .orderBy() ordena dentro de cada grupo

# Ventana por region, ordenada por monto descendente
ventana_region = Window.partitionBy("region").orderBy(F.desc("monto"))

# Aplicar ROW_NUMBER sobre la ventana
ventas_con_ranking = ventas.withColumn(
    "ranking_region",
    F.row_number().over(ventana_region)
)

print("Ventas con ranking por region:")
ventas_con_ranking.orderBy("region", "ranking_region").show()

---
# === SECCIÓN 2 ===
## 2. Funciones de Ranking

### Explicación Conceptual
- **ROW_NUMBER()**: Número único secuencial (1, 2, 3...)
- **RANK()**: Con empates, salta números (1, 2, 2, 4)
- **DENSE_RANK()**: Con empates, no salta (1, 2, 2, 3)
- **PERCENT_RANK()**: Ranking como porcentaje (0 a 1)
- **NTILE(n)**: Divide en n grupos iguales

In [ ]:
# Comparar diferentes tipos de ranking
# Crear datos con empates
datos_ranking = spark.createDataFrame([
    ("A", 100), ("B", 100), ("C", 90), 
    ("D", 90), ("E", 80), ("F", 70)
], ["nombre", "puntaje"])

ventana = Window.orderBy(F.desc("puntaje"))

comparacion = datos_ranking.select(
    "nombre",
    "puntaje",
    F.row_number().over(ventana).alias("row_number"),
    F.rank().over(ventana).alias("rank"),
    F.dense_rank().over(ventana).alias("dense_rank"),
    F.round(F.percent_rank().over(ventana), 2).alias("percent_rank"),
    F.ntile(3).over(ventana).alias("ntile_3")
)

print("Comparacion de funciones de ranking:")
comparacion.show()

In [ ]:
# Ranking por vendedor dentro de cada region
ventana_vendedor = Window.partitionBy("region").orderBy(F.desc("monto"))

top_vendedores = ventas \
    .groupBy("region", "vendedor") \
    .agg(F.sum("monto").alias("total_ventas")) \
    .withColumn("ranking", F.rank().over(
        Window.partitionBy("region").orderBy(F.desc("total_ventas"))
    ))

print("Top vendedores por region:")
top_vendedores.show()

---
# === SECCIÓN 3 ===
## 3. Funciones de Navegación

### Explicación Conceptual
- **LAG(col, n)**: Valor de n filas ANTES
- **LEAD(col, n)**: Valor de n filas DESPUÉS
- **FIRST(col)**: Primer valor de la ventana
- **LAST(col)**: Último valor de la ventana

In [ ]:
# LAG y LEAD para comparar con valores anteriores/siguientes
ventana_fecha = Window.partitionBy("vendedor").orderBy("fecha")

comparacion_temporal = ventas.select(
    "vendedor",
    "fecha",
    "monto",
    F.lag("monto", 1).over(ventana_fecha).alias("monto_anterior"),
    F.lead("monto", 1).over(ventana_fecha).alias("monto_siguiente"),
    (F.col("monto") - F.lag("monto", 1).over(ventana_fecha)).alias("cambio")
)

print("Comparacion con dia anterior/siguiente:")
comparacion_temporal.show()

In [ ]:
# FIRST y LAST
ventana_completa = Window.partitionBy("vendedor").orderBy("fecha") \
    .rowsBetween(Window.unboundedPreceding, Window.unboundedFollowing)

primero_ultimo = ventas.select(
    "vendedor",
    "fecha",
    "monto",
    F.first("monto").over(ventana_completa).alias("primera_venta"),
    F.last("monto").over(ventana_completa).alias("ultima_venta")
)

print("Primera y ultima venta por vendedor:")
primero_ultimo.show()

---
# === SECCIÓN 4 ===
## 4. Agregaciones con Ventana

### Explicación Conceptual
Las funciones de agregación (SUM, AVG, COUNT, etc.) pueden usarse con ventanas para calcular totales acumulados, promedios móviles, etc.

In [ ]:
# Suma acumulada (running total)
ventana_acumulada = Window.partitionBy("vendedor").orderBy("fecha") \
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

acumulados = ventas.select(
    "vendedor",
    "fecha",
    "monto",
    F.sum("monto").over(ventana_acumulada).alias("acumulado"),
    F.count("*").over(ventana_acumulada).alias("num_ventas"),
    F.round(F.avg("monto").over(ventana_acumulada), 2).alias("promedio_hasta_hoy")
)

print("Metricas acumuladas:")
acumulados.orderBy("vendedor", "fecha").show()

In [ ]:
# Promedio movil (ultimas 2 filas)
ventana_movil = Window.partitionBy("vendedor").orderBy("fecha") \
    .rowsBetween(-1, 0)  # Fila actual y 1 anterior

promedio_movil = ventas.select(
    "vendedor",
    "fecha",
    "monto",
    F.round(F.avg("monto").over(ventana_movil), 2).alias("promedio_movil_2d")
)

print("Promedio movil de 2 dias:")
promedio_movil.orderBy("vendedor", "fecha").show()

In [ ]:
# Comparar con el total del grupo
ventana_total = Window.partitionBy("region")

vs_total = ventas \
    .groupBy("region", "vendedor") \
    .agg(F.sum("monto").alias("ventas_vendedor")) \
    .withColumn("ventas_region", F.sum("ventas_vendedor").over(ventana_total)) \
    .withColumn("porcentaje", 
        F.round(F.col("ventas_vendedor") / F.col("ventas_region") * 100, 2)
    )

print("Participacion por vendedor en su region:")
vs_total.show()

---
# === SECCIÓN 5 ===
## 5. Frames de Ventana Personalizados

### Explicación Conceptual
El **frame** define exactamente qué filas se incluyen en el cálculo:
- `rowsBetween(start, end)`: Por número de filas
- `rangeBetween(start, end)`: Por rango de valores

**Valores especiales:**
- `Window.unboundedPreceding`: Desde el inicio
- `Window.unboundedFollowing`: Hasta el final
- `Window.currentRow`: Fila actual (0)

In [ ]:
# Diferentes tipos de frames
base = Window.partitionBy("vendedor").orderBy("fecha")

# Frame: Todas las filas anteriores hasta la actual
acumulado = base.rowsBetween(Window.unboundedPreceding, Window.currentRow)

# Frame: 1 anterior + actual + 1 siguiente (ventana de 3)
ventana_3 = base.rowsBetween(-1, 1)

# Frame: Solo las 2 filas anteriores (no incluye actual)
anteriores = base.rowsBetween(-2, -1)

resultado = ventas.select(
    "vendedor", "fecha", "monto",
    F.sum("monto").over(acumulado).alias("acumulado"),
    F.round(F.avg("monto").over(ventana_3), 0).alias("prom_ventana_3"),
    F.sum("monto").over(anteriores).alias("suma_2_anteriores")
)

print("Diferentes frames de ventana:")
resultado.filter(F.col("vendedor") == "Ana").show()

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio W.1: Ranking y Percentiles

Para cada vendedor, calcula:
1. Su ranking global por total de ventas
2. Su ranking dentro de su región
3. En qué cuartil se encuentra (NTILE 4)

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio W.1

In [ ]:
# Agregar ventas por vendedor
ventas_vendedor = ventas.groupBy("region", "vendedor") \
    .agg(F.sum("monto").alias("total"))

# Definir ventanas
ventana_global = Window.orderBy(F.desc("total"))
ventana_region = Window.partitionBy("region").orderBy(F.desc("total"))

resultado = ventas_vendedor.select(
    "region",
    "vendedor",
    "total",
    F.rank().over(ventana_global).alias("ranking_global"),
    F.rank().over(ventana_region).alias("ranking_region"),
    F.ntile(4).over(ventana_global).alias("cuartil")
)

resultado.orderBy("ranking_global").show()

### 🎯 Ejercicio W.2: Análisis de Tendencia

Para cada venta:
1. Calcula el cambio porcentual vs el día anterior
2. Indica si fue "MEJORA", "BAJA" o "PRIMERA" (sin dato anterior)
3. Calcula el promedio móvil de 2 días

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio W.2

In [ ]:
ventana = Window.partitionBy("vendedor").orderBy("fecha")
ventana_movil = ventana.rowsBetween(-1, 0)

tendencia = ventas.select(
    "vendedor", "fecha", "monto",
    F.lag("monto").over(ventana).alias("monto_anterior"),
    F.round(
        (F.col("monto") - F.lag("monto").over(ventana)) / 
        F.lag("monto").over(ventana) * 100, 1
    ).alias("cambio_pct"),
    F.when(F.lag("monto").over(ventana).isNull(), "PRIMERA")
     .when(F.col("monto") > F.lag("monto").over(ventana), "MEJORA")
     .otherwise("BAJA").alias("tendencia"),
    F.round(F.avg("monto").over(ventana_movil), 0).alias("prom_movil_2d")
)

tendencia.orderBy("vendedor", "fecha").show()

### 🎯 Ejercicio W.3: Top N por Grupo

Encuentra las 2 mejores ventas (por monto) de cada región.

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio W.3

In [ ]:
ventana_ranking = Window.partitionBy("region").orderBy(F.desc("monto"))

top_2_region = ventas \
    .withColumn("ranking", F.row_number().over(ventana_ranking)) \
    .filter(F.col("ranking") <= 2) \
    .select("region", "vendedor", "fecha", "monto", "ranking")

print("Top 2 ventas por region:")
top_2_region.orderBy("region", "ranking").show()

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **Window**: `partitionBy()` + `orderBy()` define la ventana
- **Ranking**: `row_number`, `rank`, `dense_rank`, `ntile`
- **Navegación**: `lag`, `lead`, `first`, `last`
- **Agregaciones**: `sum`, `avg`, `count` sobre ventanas
- **Frames**: `rowsBetween()`, `rangeBetween()` para personalizar

### Conexión con AWS
- **Athena/Redshift**: Funciones de ventana SQL idénticas
- **QuickSight**: Cálculos de tabla similares

### Siguiente Paso
Continúa con: `06_optimization_techniques.ipynb` para optimizar Spark